## Selección de modelo

Este notebook se centra en la evaluación y comparación de una variedad de modelos de machine learning para identificar el más adecuado para nuestro conjunto de datos específico. Tras haber realizado un extenso preprocesamiento de datos en notebooks anteriores, este notebook representa la fase crítica de selección del modelo.

Antes de nada vamos a responder a dos preguntas planteadas en las sesiones,que es un modelo GLM?Se puede aplicar a nuestros datos?

Un Modelo Lineal Generalizado (GLM) es una herramienta estadística y de machine learning que amplía los modelos lineales tradicionales para adaptarse a una variedad más amplia de tipos de datos. Esta flexibilidad se logra a través de tres componentes principales: una distribución de probabilidad para la variable dependiente que no está limitada a la distribución normal, un predictor lineal que es una suma ponderada de tus variables independientes, y una función de enlace que conecta el predictor lineal con la media de la distribución de la variable dependiente. Esta estructura hace que los GLM sean especialmente útiles para modelar diferentes tipos de variables dependientes, como las binarias, de conteo o continuas pero con distribuciones no normales.

Considerando la aplicabilidad y versatilidad de los GLM para diversos tipos de datos, en tnuestro casoso específico, donde necesimosas modelar una respuesta binaria (fraude o no fraude),emos s optado por la regresión logística, un tipo específico de GLM. La regresión logística es ideal para modelar variables dependientes binarias debido a su función de enlace logística, que vincula las probabilidades de los resultados binarios con las variables independientes. Así, en el contexto de la detección de fraude, donde los resultados son 'fraude' (1) y 'no fraude' (0), la regresión logística no solo es una elección adecuada, sino que también proporciona insights valiosos sobre cómo diferentes factores contribuyen a la probabilidad de ocurrencia de fraude
.








Con esto, se han seleccionado los siguientes modelos debido a su popularidad, versatilidad y rendimiento comprobado en una variedad de problemas de clasificación:

1. Regresión Lineal: Un modelo fundamental en estadística y machine learning, utilizado para predecir una variable objetivo continua basándose en una o más variables predictoras independientes.

2. Gaussian Naive Bayes (GaussianNB): Un clasificador simple y efectivo basado en la aplicación del teorema de Bayes con una suposición de independencia entre las características.

3. Decision Tree Classifier: Un modelo que crea un árbol de decisiones basado en la división de datos usando reglas simples, útil por su facilidad de interpretación.

4. Random Forest Classifier: Una mejora del árbol de decisiones que utiliza el enfoque de ensamblaje para mejorar la robustez y precisión.

5. LightGBM Classifier (LGBMClassifier): Una implementación eficiente de gradient boosting que es popular por su velocidad y eficiencia en conjuntos de datos grandes.

6. AdaBoost Classifier: Un algoritmo de boosting que ajusta iterativamente los pesos de los clasificadores débiles para convertirlos en un clasificador fuerte.

7. Gradient Boosting Classifier: Un modelo que construye un ensamblaje de árboles de decisión de manera secuencial, corrigiendo los errores de los árboles anteriores.

8. XGBoost Classifier (XGBClassifier): Una implementación optimizada del gradient boosting que ha ganado popularidad por su rendimiento en competiciones de datos.

9. Support Vector Classifier (SVC): Un clasificador robusto y versátil que utiliza el concepto de márgenes y vectores de soporte para la clasificación.

Inicialmente, aplicaremos estos modelos sin ajustes de hiperparámetros para evaluar su rendimiento básico. El objetivo es obtener una comparación inicial que nos permita seleccionar el modelo más prometedor, que luego será sometido a una afinación más detallada de sus hiperparámetros

Se utilizarán métricas estándar como precisión, recall, F1-score, y área bajo la curva ROC (AUC) para evaluar y comparar el rendimiento de cada modelo.



En primer lugar importamos las librerías que vamos a necesitar

In [1]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pickle
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB


Ponemos una semilla para aquellos modelos que se basen en la aelatoriedad y de esta forma asegurar la reproducibilidad.

In [2]:
seed = 42

Importamos los pickles que vamos a necesitar

In [3]:
with open('../data/X_train_val.pickle', 'rb') as file:
    X_train_val = pickle.load(file)

with open('../data/y_train_val.pickle', 'rb') as file:
    y_train_val = pickle.load(file)

with open('../data/X_val.pickle', 'rb') as file:
    X_val = pickle.load(file)

with open('../data/y_val.pickle', 'rb') as file:
    y_val = pickle.load(file)

with open('../data/X_train_nb_lr_svc_val.pickle', 'rb') as file:
    X_train_nb_lr_svc_val = pickle.load(file)

with open('../data/y_train_nb_lr_svc_val.pickle', 'rb') as file:
    y_train_nb_lr_svc_val = pickle.load(file)

with open('../data/X_val_nb_lr_svc.pickle', 'rb') as file:
    X_val_nb_lr_svc = pickle.load(file)

with open('../data/y_val_nb_lr_svc.pickle', 'rb') as file:
    y_val_nb_lr_svc = pickle.load(file)



Comprobamos que el pickle se cargo bien escogiendo X_val

In [4]:
X_val

,cat__payment_type_AA,cat__payment_type_AB,cat__payment_type_AC,cat__payment_type_AD,cat__payment_type_AE,cat__employment_status_CA,cat__employment_status_CB,cat__employment_status_CC,cat__employment_status_CD,cat__employment_status_CE,...,num__velocity_4w,num__bank_branch_count_8w,num__date_of_birth_distinct_emails_4w,num__credit_risk_score,num__bank_months_count,num__session_length_in_minutes,num__device_distinct_emails_8w,num__month,num__income_Risk_Score,num__email_score
521480,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.185084,-0.387853,0.893181,-0.588347,-0.678264,-0.180303,-0.107633,-1.488237,-0.628914,0.613710
433493,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.529661,-0.400903,0.495925,-0.875270,-0.678264,0.178818,-0.107633,-0.583015,-0.333172,1.327504
568362,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.597660,-0.383503,-0.298587,0.874962,1.166261,-0.748754,-0.107633,0.774818,-0.222268,1.327504
686926,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.027498,-0.318254,-0.695843,-0.373155,1.442940,-0.043194,-0.107633,0.322207,0.332249,1.327504
771496,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.215062,-0.383503,1.290437,-0.588347,1.719619,-0.565062,-0.107633,-0.130404,-0.628914,-0.100084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468158,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-1.953300,-0.381328,-1.291727,-0.688770,-1.047169,-0.530503,-0.107633,1.680040,-0.481043,-0.100084
606860,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.022211,-0.398728,-0.099959,0.688462,0.243999,1.342164,-0.107633,-0.130404,0.886766,-0.100084
316664,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.673099,-0.394378,0.893181,1.090155,-0.217133,-0.387298,-0.107633,-0.130404,-0.591946,-1.527672
207286,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.052715,-0.303029,-0.099959,-0.444885,0.797356,-0.358003,-0.107633,0.322207,-0.407107,0.613710


Realizamos la evaluación de tres modelos diferentes de clasificación. Estos modelos son Logistic Regression, Gaussian Naive Bayes y Support Vector Classifier (SVC). El proceso se lleva a cabo en varios pasos:



Se crea una lista classifiers_regresion que contiene tres modelos de clasificación, cada uno con sus configuraciones específicas. Por ejemplo, LogisticRegression se configura con un máximo de 3000 iteraciones. Ponemos ese número de iteraciones porque si ponemos menos el número de iteraciones es demasiado bajo y no permite que el algoritmo converja.



Posteriormente creamos un diccionario vacío (results__nb_lr_svc) para almacenar los resultados de la evaluación de cada modelo.

Luego, el código itera sobre cada modelo en classifiers__nb_lr_svc y realiza los siguientes pasos:

Creamos un pipeline (Pipeline) con el clasificador actual. Esto es útil para encadenar múltiples pasos de procesamiento y aplicarlos como una sola unidad.

Entrenamos el modelo (pipe.fit) usando el conjunto de entrenamiento (X_train_nb_lr_svc_val, y_train_nb_lr_svc_val).

Realizamos predicciones en el conjunto de validación (X_val_nb_lr_svc) y calcula métricas de rendimiento.


Para cada modelo, el código calcula y muestra:

El puntaje ROC-AUC que es una medida del rendimiento del modelo en la clasificación de clases positivas y negativas.
La precisión (accuracy), que indica la proporción general de predicciones correctas.
Un informe de clasificación (classification_report), que incluye métricas como precisión, recall y F1-score para cada clase.


Los resultados de cada modelo, incluyendo el modelo entrenado, ROC-AUC, precisión y el informe de clasificación, se almacenan en el diccionario results__nb_lr_svc.

Finalmente, los resultados se guardan en un archivo llamado model_results_nb_lr_svc usando el módulo pickle. Esto permite la persistencia de los resultados para su uso posterior, como cargar los modelos para análisis adicionales o para hacer predicciones en nuevos datos.

Hacemos estos tres modelos juntos ya que son modelos que no son árboles de decisión ni modelos ensembled y por lo tanto vamos a utilizar
datos que contienen menos variables. Esto es debido a que cuando se hizo ingienería de variables se borraron las variables originales con las que se crearon las nuevas variables para así evitar multicolinealidad. 

Los modelos de árboles de decisión y ensembled son capaces de manejar internamente esta situacion y por ello ejecutaremos esos modelos por separado con datos distintos.

In [5]:
classifiers__nb_lr_svc = [
    LogisticRegression(max_iter=3000,random_state=seed),
    GaussianNB(),
    SVC(random_state=seed)   
]

results__nb_lr_svc = {}

for classifier in classifiers__nb_lr_svc:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_nb_lr_svc_val, y_train_nb_lr_svc_val) 
    
    predictions = pipe.predict(X_val_nb_lr_svc)

    if hasattr(classifier, "predict_proba"):
        probabilities = pipe.predict_proba(X_val_nb_lr_svc)[:, 1]
        roc_auc = roc_auc_score(y_val_nb_lr_svc, probabilities)
    else:
        roc_auc = "N/A"

    accuracy = accuracy_score(y_val_nb_lr_svc, predictions)
    class_report = classification_report(y_val_nb_lr_svc, predictions)

    print(f"\n{classifier}")
    print(f"ROC-AUC score of the model: {roc_auc}")
    print(f"Accuracy of the model: {accuracy:.5f}")
    print("Classification report: ")
    print(class_report)

    results__nb_lr_svc[classifier] = {
        'model': pipe,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
        'class_report': class_report
    }

with open('../models/model_results_nb_lr_svc.pickle', 'wb') as handle:
    pickle.dump(results__nb_lr_svc, handle, protocol=pickle.HIGHEST_PROTOCOL)



LogisticRegression(max_iter=3000, random_state=42)
ROC-AUC score of the model: 0.8705586367892773
Accuracy of the model: 0.97532
Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.98      0.99    158235
           1       0.15      0.28      0.20      1765

    accuracy                           0.98    160000
   macro avg       0.57      0.63      0.59    160000
weighted avg       0.98      0.98      0.98    160000


GaussianNB()
ROC-AUC score of the model: 0.8139538469291783
Accuracy of the model: 0.79098
Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.79      0.88    158235
           1       0.04      0.73      0.07      1765

    accuracy                           0.79    160000
   macro avg       0.52      0.76      0.48    160000
weighted avg       0.99      0.79      0.87    160000


SVC(random_state=42)
ROC-AUC score of the model: N/A
Accuracy of the model:

Realizamos lo mismo pero con los algoritmos de árboles de decisión y emsembled con los datos que conservan las variables originales.

In [6]:
classifiers = [
    DecisionTreeClassifier(random_state=seed),
    RandomForestClassifier(random_state=seed),
    LGBMClassifier(random_state=seed),
    AdaBoostClassifier(random_state=seed),
    GradientBoostingClassifier(random_state=seed),
    XGBClassifier(random_state=seed)
]

results = {}

for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_val, y_train_val) 
    
    predictions = pipe.predict(X_val)

    if hasattr(classifier, "predict_proba"):
        probabilities = pipe.predict_proba(X_val)[:, 1]
        roc_auc = roc_auc_score(y_val, probabilities)
    else:
        roc_auc = "N/A"

    accuracy = accuracy_score(y_val, predictions)
    class_report = classification_report(y_val, predictions)

    print(f"\n{classifier}")
    print(f"ROC-AUC score of the model: {roc_auc}")
    print(f"Accuracy of the model: {accuracy:.5f}")
    print("Classification report: ")
    print(class_report)

    results[classifier] = {
        'model': pipe,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
        'class_report': class_report
    }

with open('../models/model_resultsArbolesYEmsembled.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)



DecisionTreeClassifier(random_state=42)
ROC-AUC score of the model: 0.6445933814329836
Accuracy of the model: 0.91316
Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.92      0.95    158235
           1       0.05      0.37      0.09      1765

    accuracy                           0.91    160000
   macro avg       0.52      0.64      0.52    160000
weighted avg       0.98      0.91      0.94    160000


RandomForestClassifier(random_state=42)
ROC-AUC score of the model: 0.8832935934298601
Accuracy of the model: 0.98199
Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    158235
           1       0.21      0.23      0.22      1765

    accuracy                           0.98    160000
   macro avg       0.60      0.61      0.60    160000
weighted avg       0.98      0.98      0.98    160000

[LightGBM] [Info] Number of positive: 7058, number of negati

Leemos y cargamos los datos de los archivo pickle que hemos guardado anteriormente.

In [7]:
with open('../models/model_resultsArbolesYEmsembled.pickle', 'rb') as handle:
    results = pickle.load(handle)

In [8]:
with open('../models/model_results_nb_lr_svc.pickle', 'rb') as handle:
    results_nb_lr_svc = pickle.load(handle)

Imprimimos de manera ordenada y detallada el rendimiento de varios modelos de clasificación que han sido previamente evaluados y cuyos resultados se han almacenado en los diccionarios resultsRegresion y results. De esta forma leyendo los pickles y ejecutando estos bucles podemos evaluar y ver las distintas métricas del modelo sin necesidad de volver a entrenar los modelos.

In [9]:
for classifier, metrics in results_nb_lr_svc.items():
    classifier_name = classifier.__class__.__name__
    
    print(f"\nModelo: {classifier_name}")
    print(f"ROC-AUC score of the model: {metrics['roc_auc']}")
    print(f"Accuracy of the model: {metrics['accuracy']:.5f}")
    print("Classification report: ")
    print(metrics['class_report'])


Modelo: LogisticRegression
ROC-AUC score of the model: 0.8705586367892773
Accuracy of the model: 0.97532
Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.98      0.99    158235
           1       0.15      0.28      0.20      1765

    accuracy                           0.98    160000
   macro avg       0.57      0.63      0.59    160000
weighted avg       0.98      0.98      0.98    160000


Modelo: GaussianNB
ROC-AUC score of the model: 0.8139538469291783
Accuracy of the model: 0.79098
Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.79      0.88    158235
           1       0.04      0.73      0.07      1765

    accuracy                           0.79    160000
   macro avg       0.52      0.76      0.48    160000
weighted avg       0.99      0.79      0.87    160000


Modelo: SVC
ROC-AUC score of the model: N/A
Accuracy of the model: 0.98438
Classification rep

In [10]:
for classifier_name, data in results.items():
    print(f"\nModelo: {classifier_name}")
    print(f"ROC-AUC score of the model: {data['roc_auc']}")
    print(f"Accuracy of the model: {data['accuracy']:.5f}")
    print("Classification report: ")
    print(data['class_report'])


Modelo: DecisionTreeClassifier(random_state=42)
ROC-AUC score of the model: 0.6445933814329836
Accuracy of the model: 0.91316
Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.92      0.95    158235
           1       0.05      0.37      0.09      1765

    accuracy                           0.91    160000
   macro avg       0.52      0.64      0.52    160000
weighted avg       0.98      0.91      0.94    160000


Modelo: RandomForestClassifier(random_state=42)
ROC-AUC score of the model: 0.8832935934298601
Accuracy of the model: 0.98199
Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    158235
           1       0.21      0.23      0.22      1765

    accuracy                           0.98    160000
   macro avg       0.60      0.61      0.60    160000
weighted avg       0.98      0.98      0.98    160000


Modelo: LGBMClassifier(random_state=42)
ROC

Con esta información en el siguiente notebook decediremos cual es el modelo más óptimo y porqué.